In [1]:
# Imports the cffi wrapper for the sciencemode library
from sciencemode import sciencemode
import time

## Creating same basic variables for the sciencemode library

In [27]:
ack = sciencemode.ffi.new("Smpt_ack*")
device = sciencemode.ffi.new("Smpt_device*")
extended_version_ack = sciencemode.ffi.new("Smpt_get_extended_version_ack*")
ml_init = sciencemode.ffi.new("Smpt_ml_init*")
ml_update = sciencemode.ffi.new("Smpt_ml_update*")
ml_get_current_data = sciencemode.ffi.new("Smpt_ml_get_current_data*")
ml_get_current_data_ack = sciencemode.ffi.new("Smpt_ml_get_current_data_ack*")

## Identifying and Opening the P24 com port 

In [47]:
found_p24 = False

for i in range(15):
    if found_p24:
        continue
    com_str = "COM%d" % (i + 1)
    com = sciencemode.ffi.new("char[]", com_str.encode("ascii"))
    ret = sciencemode.smpt_check_serial_port(com)
    if (ret):
        print("Port port %s exists" % com_str)
        ret = sciencemode.smpt_open_serial_port(device,com)
        if not ret:
            ret = sciencemode.smpt_close_serial_port(device)
            continue
        ret = sciencemode.smpt_send_get_extended_version(device, 0)
        if not ret:
            ret = sciencemode.smpt_close_serial_port(device)
            continue
        cnt = 0
        while ( not sciencemode.smpt_new_packet_received(device) and cnt < 3):
            time.sleep(1)
        if cnt == 3:
            ret = sciencemode.smpt_close_serial_port(device)
            continue
        sciencemode.smpt_last_ack(device, ack);
        if ack.command_number != sciencemode.Smpt_Cmd_Get_Extended_Version_Ack:
            ret = sciencemode.smpt_close_serial_port(device)
            continue
        else:
            found_p24 = True
            ret = sciencemode.smpt_get_get_extended_version_ack(device, extended_version_ack)
if found_p24:
    print("Found and opened a stimulator at %s" % com_str)
    print("Firmware %d.%d.%d" % (extended_version_ack.uc_version.fw_version.major, extended_version_ack.uc_version.fw_version.minor, extended_version_ack.uc_version.fw_version.revision))
    print("SMPT %d.%d.%d" % (extended_version_ack.uc_version.smpt_version.major, extended_version_ack.uc_version.smpt_version.minor, extended_version_ack.uc_version.smpt_version.revision))
    

Port port COM6 exists
Port port COM7 exists
Port port COM10 exists
Found and opened a stimulator at COM10
Firware 1.0.0
SMPT 4.0.0


## Initializing Mid Level and stimulating all 8 channels for 10 seconds

In [45]:
# Cleaning up received packages
while ( sciencemode.smpt_new_packet_received(device)):
    sciencemode.smpt_clear_ack(ack)
    sciencemode.smpt_last_ack(device, ack)

ml_init.packet_number = sciencemode.smpt_packet_number_generator_next(device)
ret = sciencemode.smpt_send_ml_init(device, ml_init)
print("smpt_send_ml_init: ", ret)
time.sleep(1)

ml_update.packet_number = sciencemode.smpt_packet_number_generator_next(device)
for i in range(8):
    ml_update.enable_channel[i] = True
    ml_update.channel_config[i].period = 20
    ml_update.channel_config[i].number_of_points = 3
    ml_update.channel_config[i].points[0].time = 100
    ml_update.channel_config[i].points[0].current = 20
    ml_update.channel_config[i].points[1].time = 100
    ml_update.channel_config[i].points[1].current = 0
    ml_update.channel_config[i].points[2].time = 100
    ml_update.channel_config[i].points[2].current = -20
    
ret = sciencemode.smpt_send_ml_update(device, ml_update)
print("smpt_send_ml_update: ", ret)

for i in range(10):
    ml_get_current_data.data_selection = sciencemode.Smpt_Ml_Data_Channels
    ml_get_current_data.packet_number = sciencemode.smpt_packet_number_generator_next(device)
    ret = sciencemode.smpt_send_ml_get_current_data(device, ml_get_current_data)
    time.sleep(1)
    while ( sciencemode.smpt_new_packet_received(device)):
        sciencemode.smpt_clear_ack(ack)
        sciencemode.smpt_last_ack(device, ack)
        if (ack.command_number != sciencemode.Smpt_Cmd_Ml_Get_Current_Data_Ack):
            continue
        ret = sciencemode.smpt_get_ml_get_current_data_ack(device, ml_get_current_data_ack)
        if (not ret):
            print("smpt_get_ml_get_current_data_ack: ", ret)
        error_on_channel = False
        for i in range(8):
            if (ml_get_current_data_ack.channel_data.channel_state[i] != sciencemode.Smpt_Ml_Channel_State_Ok):
                error_on_channel = True
        if error_on_channel:
            print("Error on channel")
        else:
            print("All channels ok")
    

packet_number = sciencemode.smpt_packet_number_generator_next(device)
ret = sciencemode.smpt_send_ml_stop(device, packet_number)
print("smpt_send_ml_stop: ", ret)

smpt_send_ml_init:  True
smpt_send_ml_update:  True
All channels ok
All channels ok
All channels ok
All channels ok
All channels ok
All channels ok
All channels ok
All channels ok
All channels ok
All channels ok
smpt_send_ml_stop:  True


## Closing the opened device port

In [46]:
ret = sciencemode.smpt_close_serial_port(device)
print("smpt_close_serial_port: ", ret)

smpt_close_serial_port:  True
